# Creating a connectivity table of the auditory neurons
Sent from Sama's Lab. Ian Quah sent the auditory_connectivity_table and unique_ids. We read those zip and json files and save the results as txt file tables to be easily read back in as dataframes. There are almost 22,000 unique auditory neurons in these tables. 

In [1]:
import pandas as pd
import numpy as np
import json
_auditory_neuron_conns = pd.read_csv("auditory_neuron_connections.csv.gz", compression='gzip')

auditory_neuron_conns = _auditory_neuron_conns.rename(columns={"Unnamed: 0": "index_in_connections_table"})

with open("unique_neuron_ids.json", "r") as f:
    unique_ids = json.load(f)

In [2]:
auditory_neuron_conns

,index_in_connections_table,pre_root_id,post_root_id,neuropil,syn_count,nt_type
0,0,720575940629970489,720575940631267655,AVLP_R,7,GABA
1,4,720575940616871878,720575940621203973,AVLP_L,13,GABA
2,6,720575940659399809,720575940604569824,AVLP_R,158,ACH
3,9,720575940619615552,720575940613174963,AVLP_R,5,ACH
4,10,720575940651461622,720575940612290019,AVLP_R,10,GLUT
...,...,...,...,...,...,...
692004,3869845,720575940634453093,720575940626663953,AMMC_L,3,ACH
692005,3869854,720575940624999114,720575940627970971,AVLP_R,1,ACH
692006,3869860,720575940628879055,720575940620502110,AVLP_L,4,ACH
692007,3869866,720575940612446866,720575940606567125,SAD,3,ACH


In [3]:
len(unique_ids)

21942

In [4]:
df_edgelist = auditory_neuron_conns[["pre_root_id", "post_root_id", "syn_count"]]

In [7]:
df_edgelist.to_csv("auditory_connections.txt", index=False, header=False, sep=",")

In [9]:
# Create a connectivity matrix
vals = np.unique(auditory_neuron_conns[["pre_root_id", 'post_root_id']])
vals

array([720575940600433181, 720575940600646173, 720575940602120928, ...,
       720575940661329281, 720575940661335681, 720575940661338497])

In [10]:
# Compare vals and unique_ids
print(len(vals))
print(len(unique_ids))


# compare the ids between the two
print(set(vals) - set(unique_ids))
print(set(unique_ids) - set(vals))

21942
21942
set()
set()


## Select connections for tuned neurons only

In [ ]:
# Some auditory neurons with selectivity labels provided by Ian from Sama's lab
train_dat = pd.read_csv("data/train_dat.csv")
train_dat

,Unnamed: 0,Flywire 2024 ID,Neuropils,preference,# imaged females,# responding females,prop females with auditory responses
0,0,720575940619783518,AVLP_pr01,sine,4,4,1.000000
1,1,720575940620062633,AVLP_pr01,sine,4,4,1.000000
2,2,720575940602914988,AVLP_pr01,sine,4,4,1.000000
3,3,720575940620195719,AVLP_pr01,sine,4,4,1.000000
4,4,720575940634508174,AVLP_pr01,sine,4,4,1.000000
...,...,...,...,...,...,...,...
398,398,720575940655398817,GNG_pr01,sine,17,4,0.235294
399,399,720575940630203589,GNG_pr01,sine,17,4,0.235294
400,400,720575940617641077,GNG_pr01,sine,17,4,0.235294
401,401,720575940612801022,GNG_pr01,sine,17,4,0.235294


In [13]:
# get connections only for tuned neurons in train_dat
tuned_neurons = train_dat["Flywire 2024 ID"]
tuned_neuron_conns = auditory_neuron_conns[auditory_neuron_conns["pre_root_id"].isin(tuned_neurons) & auditory_neuron_conns["post_root_id"].isin(tuned_neurons)]
#tuned_neuron_conns = tuned_neuron_conns[["pre_root_id", "post_root_id", "syn_count"]]

In [14]:
tuned_neuron_conns

,index_in_connections_table,pre_root_id,post_root_id,neuropil,syn_count,nt_type
105,421,720575940630302551,720575940630396790,AVLP_R,16,SER
600,2236,720575940631153194,720575940626258690,AVLP_R,38,ACH
5420,19860,720575940637175349,720575940654167201,AVLP_R,6,ACH
8707,31535,720575940610638434,720575940634299071,SAD,1,ACH
9415,34005,720575940606743170,720575940615723077,AMMC_R,5,GABA
...,...,...,...,...,...,...
691431,3866907,720575940642412941,720575940644968302,WED_R,2,ACH
691640,3867995,720575940604839200,720575940644968302,SAD,5,ACH
691687,3868264,720575940606332629,720575940637410394,WED_L,2,ACH
691732,3868485,720575940620694015,720575940627350060,AVLP_R,5,ACH


In [ ]:
# export to .txt for modularity analysis
tuned_neuron_conns[["pre_root_id", "post_root_id", "syn_count"]].to_csv("gcm_pipeline/tuned_auditory_conns.txt", index=False, header=False, sep=",")

## Make dataframe of unique auditory neurons
Making this so that we can export as a table and fill in the response selectivties by hand into a new column based on the Baker paper. The consolidated_cell_types.csv file has types matched with ids but there are almost 2,000 missing rows and I'm not sure why. We'll have to do what we can with it. There will be some auditory neurons with NaN in their type columns. 

In [ ]:
# I don't understand these names yet, but they are not the cell types
names = pd.read_csv("/Users/ggutierr/Documents/FlyWireData/names.csv")
names

,root_id,name,group
0,720575940596125868,LO.LOP.561,LO.LOP
1,720575940597856265,ME.2982,ME
2,720575940597944841,ME.LO.7713,ME.LO
3,720575940598267657,ME.4244,ME
4,720575940599333574,ME.5182,ME
...,...,...,...
139250,720575940661335681,GNG.962,GNG
139251,720575940661336193,ME.LO.500,ME.LO
139252,720575940661337217,SCL.MB_ML.12,SCL.MB_ML
139253,720575940661338497,GNG.878,GNG


In [40]:
# cell types from FlyWire
types = pd.read_csv("/Users/ggutierr/Documents/FlyWireData/consolidated_cell_types.csv")
#types.rename(columns={"root_id": "id"}, inplace=True)
types

,root_id,primary_type,additional_type(s)
0,720575940596125868,T5c,NaN
1,720575940597856265,Tm16,NaN
2,720575940597944841,Tm7,CB3851
3,720575940598267657,TmY15,NaN
4,720575940599333574,Tm1,NaN
...,...,...,...
137682,720575940661335681,CB0180,NaN
137683,720575940661336193,TmY3,NaN
137684,720575940661337217,KCab-p,NaN
137685,720575940661338497,CB0904,NaN


In [38]:
# class and cell type info from FlyWire
cell_class = pd.read_csv("/Users/ggutierr/Documents/FlyWireData/classification.csv")
cell_class

,root_id,flow,super_class,class,sub_class,cell_type,hemibrain_type,hemilineage,side,nerve
0,720575940640144768,intrinsic,optic,NaN,L1-5,L1,NaN,NaN,right,NaN
1,720575940630759755,intrinsic,central,NaN,NaN,CB1092,NaN,LB3,left,NaN
2,720575940637932009,intrinsic,central,Kenyon_Cell,NaN,NaN,KCg-m,MBp3,left,NaN
3,720575940606131116,intrinsic,optic,NaN,NaN,Tlp4,NaN,NaN,left,NaN
4,720575940633723091,intrinsic,optic,NaN,NaN,Tm4,NaN,NaN,left,NaN
...,...,...,...,...,...,...,...,...,...,...
139250,720575940632239661,afferent,sensory,olfactory,NaN,ORN_VM6l,ORN_VC5,NaN,right,AN
139251,720575940624423312,intrinsic,optic,NaN,NaN,TmY5a,NaN,NaN,right,NaN
139252,720575940625090916,intrinsic,optic,NaN,NaN,NaN,NaN,NaN,left,NaN
139253,720575940631596103,intrinsic,optic,NaN,NaN,T2a,NaN,NaN,left,NaN


In [39]:
cell_class[cell_class['root_id'].isin(unique_ids)]

,root_id,flow,super_class,class,sub_class,cell_type,hemibrain_type,hemilineage,side,nerve
1,720575940630759755,intrinsic,central,NaN,NaN,CB1092,NaN,LB3,left,NaN
5,720575940632934751,efferent,descending,NaN,NaN,DNge094,NaN,LB7,right,CV
8,720575940613793022,intrinsic,central,NaN,NaN,CB2365,NaN,VLPl1_or_VLPl5,right,NaN
12,720575940625765424,intrinsic,central,NaN,NaN,CB2056,NaN,VESa2,left,NaN
14,720575940635859354,intrinsic,central,NaN,NaN,NaN,IB008,putative_primary,left,NaN
...,...,...,...,...,...,...,...,...,...,...
139222,720575940640173184,afferent,ascending,AN,AN_GNG,AN_GNG_93,NaN,NaN,right,CV
139226,720575940620476381,intrinsic,central,NaN,NaN,NaN,VES023,PSa1,left,NaN
139239,720575940611580313,intrinsic,visual_projection,NaN,NaN,NaN,LC6,VPNd3,left,NaN
139243,720575940611090244,intrinsic,central,LHCENT,NaN,NaN,LHCENT8,primary,right,NaN


In [41]:
# grab the auditory neurons
auditory_neurons = names[names["root_id"].isin(unique_ids)]

# merge the cell types with the auditory neurons
auditory_neurons = auditory_neurons.merge(types, left_on="root_id", right_on="root_id", how="left")

# drop the id column and rename the root_id column
#auditory_neurons.drop(columns=["id"], inplace=True)
#auditory_neurons.rename(columns={"root_id": "id"}, inplace=True)

auditory_neurons

,root_id,name,group,primary_type,additional_type(s)
0,720575940600433181,AMMC.80,AMMC,JO-B,NaN
1,720575940600646173,AMMC.SAD.20,AMMC.SAD,JO-mz,NaN
2,720575940602120928,AVLP.1350,AVLP,CB1783,AVLP_pr36
3,720575940602132509,SAD.209,SAD,JO-EVM,NaN
4,720575940602309600,GNG.4231,GNG,NaN,NaN
...,...,...,...,...,...
21937,720575940661321857,PLP.538,PLP,PLP160,NaN
21938,720575940661325697,ME.PLP.76,ME.PLP,MTe07,"hb-1036546847, hb-1190339995, hb-5812983942, h..."
21939,720575940661329281,AVLP.1477,AVLP,CB2488,NaN
21940,720575940661335681,GNG.962,GNG,CB0180,NaN


In [23]:
# save merged auditory
auditory_neurons.to_csv("data/auditory_neurons.txt", index=False)

In [24]:
auditory_neurons.head(35)

,id,name,group,primary_type,additional_type(s)
0,720575940600433181,AMMC.80,AMMC,JO-B,NaN
1,720575940600646173,AMMC.SAD.20,AMMC.SAD,JO-mz,NaN
2,720575940602120928,AVLP.1350,AVLP,CB1783,AVLP_pr36
3,720575940602132509,SAD.209,SAD,JO-EVM,NaN
4,720575940602309600,GNG.4231,GNG,NaN,NaN
5,720575940602313952,GNG.3022,GNG,CB1937,NaN
6,720575940602337504,AVLP.1498,AVLP,AN_AVLP_41,NaN
7,720575940602353632,GNG.2718,GNG,LB1a,NaN
8,720575940602380768,AMMC.18,AMMC,CB1538,NaN
9,720575940602382304,GNG.1544,GNG,AN_GNG_SAD_13,NaN


In [25]:
names[names["root_id"] == 720575940602309600]

,root_id,name,group
52,720575940602309600,GNG.4231,GNG


In [27]:
types[types["root_id"] == 720575940602309600]

,id,primary_type,additional_type(s)


In [29]:
auditory_neurons.value_counts('primary_type').head(20)

primary_type
BM_InOm     922
LC12        381
LC17        276
LPLC2       210
JO-B        190
LC18        188
JO-E        179
LC9         179
LC16        151
LPLC1       140
LC21        130
LB3         128
LC11        127
LC6         125
LC15        106
LC4         104
JO-A         82
TPMN         75
mAL          68
BM_Taste     64
Name: count, dtype: int64

In [37]:
auditory_neurons.value_counts('additional_type(s)').head(20)

additional_type(s)
mAL1, mAL2A, mAL2B, mAL3A, mAL3B, mAL4, mAL5A, mAL5B, mAL6    68
pIP13                                                         52
LC31                                                          47
B1                                                            43
aSP9                                                          40
MC62                                                          37
aIP2                                                          35
PVLP005                                                       30
CL122                                                         29
AVLP_pr05                                                     25
aSP18                                                         25
aIP3                                                          25
AVLP_pr22                                                     25
SMP259, SMP260, SMP263, SMP264, SMP265                        24
B1-5                                                          23
CB3821

In [36]:
auditory_neurons[auditory_neurons["additional_type(s)"] == "AVLP_pr05"]

,id,name,group,primary_type,additional_type(s)
44,720575940603331686,WED.ICL.10,WED.ICL,CL253,AVLP_pr05
549,720575940606202891,AVLP.SMP.48,AVLP.SMP,AVLP560,AVLP_pr05
737,720575940607049941,AVLP.2772,AVLP,CB2260,AVLP_pr05
2762,720575940612965791,AVLP.2692,AVLP,CB2260,AVLP_pr05
3978,720575940615348770,AVLP.2680,AVLP,CB1660,AVLP_pr05
4184,720575940615729179,AVLP.2648,AVLP,CB1380,AVLP_pr05
4215,720575940615798045,AVLP.ICL.329,AVLP.ICL,CL252,AVLP_pr05
5045,720575940617193009,AVLP.ICL.331,AVLP.ICL,CL252,AVLP_pr05
5259,720575940617529669,AVLP.ICL.321,AVLP.ICL,CL252,AVLP_pr05
5429,720575940617842196,PLP.554,PLP,PLP160,AVLP_pr05
